# 実践その参（言語処理パート）

HugginefaceのRobertモデルを使って、ジャンルを分類し、その後OpenAI社のGPTのAPIを使って回答を生成する。

本課題は前半と後半に分かれます。

前半:
HugginefaceのRobertモデルを使って、ジャンルを分類できるモデルの定義、学習

後半:
GPTを使ってのリクエスト


想定される実際のタスク:
「コールセンターのやりとりを問い合わせとクレームの2種類に分類」し「簡単な状況説明を生成しつつスーパーバイザーに通知する」など。


## 条件
- HugginefaceのRobertaモデルを使うこと。
- 学習のループは自分で書くこと
- 生成にはOpenAI社のGPTを使うこと
- テスト用の質問に対してreazonableな回答が生成できること

このタスクではGPTをAPI経由で使う必要があります。
そのため、いくばくかの費用がかかる可能性がありますので、ご理解いただけますと幸いです。

In [ ]:
# ここにStudent IDを記載しておいてください。
Student_ID = ""

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import

import os
import numpy as np
import math
import random
import torch

import random
random.seed(55)

# 前半

## データの準備

最低でも３種類以上のテキストデータを用意します。

もし、どんなデータを使って良いかわからない場合は、青空文庫から探してみましょう。

https://www.aozora.gr.jp/

In [ ]:
# 以下はlivedoorニュースを使った場合。
# 9通りのカテゴリーが入ってます。

# # livedoorニュースのデータセットをダウンロードして、解凍します。

# ! wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
# ! tar xvzf ldcc-20140209.tar.gz

In [ ]:
# # カテゴリー一覧（例）
# category_list = [
#     "dokujo-tsushin",
#     "it-life-hack",
#     "kaden-channel",
#     "livedoor-homme",
#     "movie-enter",
#     "peachy",
#     "smax",
#     "sports-watch",
#     "topic-news",
# ]

# カテゴリーの一覧（カテゴリー名は自分で考える）
category_list = [
    "category1",
    "category2",
    "category3",
]

In [ ]:
# ディレクトリをチェックして、リストに格納します。

import glob

# read files
files1 = glob.glob('./text/*/*.txt')

files2 = []
for item in files1:
    if "LICENSE.txt" not in item:
        files2.append(item)

len(files2)

## Haggingface

本プロジェクトではHaggingfaceのソースコードを利用します。

https://github.com/huggingface/transformers


In [ ]:
# transformersをインストールします。
!pip install transformers --upgrade

In [ ]:
from transformers import AutoTokenizer, RobertaForSequenceClassification, RobertaForMaskedLM, RobertaForSequenceClassification

## トークナイザー

In [ ]:
# tokenizer
# 本当はトークナイザーも自作できた方が良いのですが、作成にはそこそこパワーのあるマシンが必要になるため、使えるものをご提供します。

# トークナイザをダウンロード
! wget https://payloadcms.shabelab.com/assets/datasets/roberta_base_tokenizer_0511.tar.gz

# ダウンロードしたトークナイザを解凍
! tar xvzf roberta_base_tokenizer_0511.tar.gz

In [ ]:
# トークナイザをインスタンス化
tokenizer = AutoTokenizer.from_pretrained("./roberta_base_tokenizer_0511")

In [ ]:
# トークナイザをテスト
tokenizer("今日の晩御飯はカレーでした。")

In [ ]:
tokenizer.decode([0, 8296, 27403, 1751, 2908, 289, 14114, 4215, 283, 2])

## Datasetの準備

ヒント動画を参考にしても良いですし、他のリソースを参考にしていただいても構いません。

バッチデータを以下のような感じで取得して情報を出力すると
```
one_batch = next(iter(train_loader))
print("one_batch count: ", len(one_batch))
print("")
print("one_batch.0.shape: ", one_batch[0].shape)
print("one_batch.1.shape: ", one_batch[1].shape)
print("one_batch.2.shape: ", one_batch[2].shape)
print("")
print("one_batch.1: ", one_batch[1])
print("one_batch.2: ", one_batch[2])
```

以下のようなデータが出力されます。
```
one_batch count:  3

one_batch.0.shape:  torch.Size([2, 512])
one_batch.1.shape:  torch.Size([2, 512])
one_batch.2.shape:  torch.Size([2, 1])

one_batch.1:  tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])
one_batch.2:  tensor([[0],
        [8]])
```

取得できるデータは順番に実際のインプットデータ、マスクデータ、そして正解ラベルとなっている必要があります。
マスクデータはblock_sizeが512でインプットするデータのトークン数が512以上の場合（512までで残りはカットされます）は512個の1がならび、512未満の場合はトークンの存在するところまでは1でそれ以外（パディングで埋められた部分）が0となります。

ところで、一つのバッチに入ってきたデータのトークン数が150と170など、いずれも最大直に満たない場合は、大きな方の値（この例では170）に揃えるようにしてください。

ヒント1: 講師はDataset型を継承したMyDatasetを定義し、バッチ化する際にcollateファンクションを使ってマスクを作成しています。  
ヒント2: transformersを使ったモデルが重たいので、バッチサイズは2くらいで十分です。

## アーキテクチャ

In [ ]:
# スクラッチでゼロから学習を行うには流石にデータセットが少ないので、pre-trainedのウェイトをダウンロードし、
# そこからファインチューニングを書けることにします。
! wget https://payloadcms.shabelab.com/assets/datasets/roberta-pretrain-0-1500000.tar.gz

In [ ]:
# ダウンロードしてきたウェイトを解凍します。
! tar xvxf roberta-pretrain-0-1500000.tar.gz

In [ ]:
# change number of lables
# num_labelsでカテゴリー数を指定します。
# 分類するドキュメントが３種類の場合は３を指定してください

# mm_config.num_labels = 9
model = RobertaForSequenceClassification.from_pretrained(
    "./roberta-pretrain-0-1500000/", num_labels=9)

model.classifier


## トレーニングループ

下記を参考に学習ループを自分で作成しましょう。  
https://huggingface.co/transformers/model_doc/roberta.html#robertaforsequenceclassification

ヒント: GoogleのColabは1時間触らないでいるとタイムアウトします。学習には最低でも数時間かかります。  
途中で止めて、そこまでのウェイトを保存、ダウンロードして、次回をそれをアップロード、読み込むことで途中から始めることも可能です。

In [ ]:
# optimizer
# 下記はadamを使っていますが、他のものを使ってもokです。

optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

In [ ]:
# device

device = torch.device("cuda:0")
model = model.to(device)


In [ ]:
# 学習ループ
# ここに for ループを作成してください。


In [ ]:
# save
model.save_pretrained("checkpoint-1")

## 前半のテスト

下記は変更の必要はありません。  
例では"sports-watch"として推論できればokです。


In [ ]:
from transformers import AutoTokenizer, RobertaForSequenceClassification, RobertaForMaskedLM

In [ ]:
model2 = model.from_pretrained("checkpoint-1")
model2.classifier

In [ ]:
# 自分で、自分で学習させたテキストのサンプルを入れてください。
# 学習データと全く同じものは避けてください。

sample ""
# sample = "前田智徳氏、走攻守そろった新人・近本光司を称賛　「ルーキーではなかなかできません」0LINE共有ボタン2019年5月23日 13時30分 Sports Watch22日放送、テレビ朝日「報道ステーション」に、野球解説者の前田智徳氏が出演。阪神タイガースの新人・近本光司を称賛した。同日の東京ヤクルトスワローズ戦で、近本は守備でチームを救った。1-1の同点で迎えた5回表、二死二塁のピンチで、見事なバックホームから失点を防いだ。危機を脱した阪神は、7回ウラに糸井嘉男のタイムリーで勝ち越し、連勝を飾っている。前田氏は近本の守備を称賛したうえで、「注目は足」と述べた。この日の近本は、1回に内野安打で出塁すると盗塁成功。5回にも三盗を決めている。盗塁数はリーグトップの13。新人で盗塁王となれば、39盗塁を記録した2001年の赤星憲広以来、18年ぶりの快挙だ。前田氏は「十分にチャンスはある」と、近本のさらなる盗塁に太鼓判を押す。さらに、前田氏は「軸足にしっかり体重を乗せて、レフト（逆方向）へ力強い打球を打てる」と、近本の打撃にも称賛。「ルーキーではなかなかできません」と、走攻守の三拍子がそろった近本が新人離れしていると賛辞を寄せた。"

input_ids = tokenizer(
        sample,
        add_special_tokens=True,
        return_tensors="pt",
        max_length=512,
        truncation=True,).input_ids

input_ids.shape


In [ ]:
resuslts = model2(input_ids=input_ids)

logits = resuslts.logits
logits.shape

In [ ]:
_index = logits.argmax().item()
_index

In [ ]:
category_list[_index]

# 後半

やりとりの作成（回答の生成）はGPTで行います

まずは、以下のコードでopenaiのライブラリをインストールしてください。

!pip install openai

以下はGPTの使用例です

```
import os
import openai
openai.api_key = "OPENAI_API_KEY"

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "あなたなアシスタントです。ドキュメントの分類を担当します"},
    {"role": "user", "content": "こんにちは！"}
  ]
)

print(completion.choices[0].message)
```

# 最終連結テスト


想定するやりとりのイメージは以下の通りです。

1. まずユーザーがメッセージを送ります。

2. データをプログラムで分類します。

3. アシスタントに 2. で分類した結果を貼り付けて質問を送ります。

4. アシスタントから回答が返ってくる

大抵の回答／分類はGPTだけでもできますが、カテゴリー名が１００％一致するわけでもなく、かつ無料でも無いので、「何についての問い合わせか」をデータベースへ保存しておく目的であれば、固定が便利です。早いし。

In [ ]:
# 1. まず質問分を作成してください。

# user_query = """
# 以下の文章を要約してもらえますか？
#
# 前田智徳氏、走攻守そろった新人・近本光司を称賛　「ルーキーではなかなかできません」0LINE共有ボタン2019年5月23日 13時30分 Sports Watch22日放送、テレビ朝日「報道ステーション」に、野球解説者の前田智徳氏が出演。阪神タイガースの新人・近本光司を称賛した。同日の東京ヤクルトスワローズ戦で、近本は守備でチームを救った。1-1の同点で迎えた5回表、二死二塁のピンチで、見事なバックホームから失点を防いだ。
# 危機を脱した阪神は、7回ウラに糸井嘉男のタイムリーで勝ち越し、連勝を飾っている。前田氏は近本の守備を称賛したうえで、「注目は足」と述べた。
# この日の近本は、1回に内野安打で出塁すると盗塁成功。5回にも三盗を決めている。盗塁数はリーグトップの13。新人で盗塁王となれば、39盗塁を記録した2001年の赤星憲広以来、18年ぶりの快挙だ。前田氏は「十分にチャンスはある」と、近本のさらなる盗塁に太鼓判を押す。さらに、前田氏は「軸足にしっかり体重を乗せて、レフト（逆方向）へ力強い打球を打てる」と、近本の打撃にも称賛。
# 「ルーキーではなかなかできません」と、走攻守の三拍子がそろった近本が新人離れしていると賛辞を寄せた。
# """

user_query = ""



In [ ]:
# 2. 分類
# 前半最後の部分でやった分類をここにも書く。関数化できているとベター

# 以下は例です
# user_query_with_cateogry = """
# 以下の文章を要約してもらえますか？

# 前田智徳氏、走攻守そろった新人・近本光司を称賛　「ルーキーではなかなかできません」0LINE共有ボタン2019年5月23日 13時30分 Sports Watch22日放送、テレビ朝日「報道ステーション」に、野球解説者の前田智徳氏が出演。阪神タイガースの新人・近本光司を称賛した。同日の東京ヤクルトスワローズ戦で、近本は守備でチームを救った。1-1の同点で迎えた5回表、二死二塁のピンチで、見事なバックホームから失点を防いだ。
# 危機を脱した阪神は、7回ウラに糸井嘉男のタイムリーで勝ち越し、連勝を飾っている。前田氏は近本の守備を称賛したうえで、「注目は足」と述べた。
# この日の近本は、1回に内野安打で出塁すると盗塁成功。5回にも三盗を決めている。盗塁数はリーグトップの13。新人で盗塁王となれば、39盗塁を記録した2001年の赤星憲広以来、18年ぶりの快挙だ。前田氏は「十分にチャンスはある」と、近本のさらなる盗塁に太鼓判を押す。さらに、前田氏は「軸足にしっかり体重を乗せて、レフト（逆方向）へ力強い打球を打てる」と、近本の打撃にも称賛。
# 「ルーキーではなかなかできません」と、走攻守の三拍子がそろった近本が新人離れしていると賛辞を寄せた。

# カテゴリー名も返してください。カテゴリー名はバックエンドシステムで使うので変更しないでください。
# カテゴリー名: 「sports-watch」
# """

user_query_with_cateogry = ""

In [ ]:
# !pip install openai

In [ ]:
# 3. GPTに回答を生成させます。


import os
import openai
openai.api_key = "＜OpenAIのキー＞"

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
        "role": "system",
        "content": "あなたなアシスタントです。"
    },
    {
        "role": "user",
        "content": user_query_with_cateogry
    },
  ]
)

print(completion.choices[0].message.content)


カテゴリー名: 「sports-watch」

野球解説者の前田智徳氏が阪神タイガースの新人・近本光司を称賛し、走攻守の三拍子がそろった近本の実績に賛辞を寄せた。近本は守備でチームを救い、盗塁でも活躍しており、新人離れしていると評価されている。
